In [1]:
stations = sc.textFile('stations.csv')
status = sc.textFile('status.csv')

In [2]:
status.take(1)

['10,7,8,"2014-12-30 15:37:02"']

In [31]:
status2 = status.map(lambda x: x.split(',')) \
.map(lambda x: (x[0], x[1], x[2], x[3].replace('"',''))) \
.map(lambda x: (x[0], x[1], x[2], x[3].split(' '))) \
.map(lambda x: (x[0], x[1], x[2], x[3][0].split('-'), x[3][1].split(':'))) \
.map(lambda x: (int(x[0]), int(x[1]), int(x[3][0]), int(x[3][1]), int(x[3][2]),
int(x[4][0])))

status2.take(5)

[(10, 7, 2014, 12, 30, 15),
 (10, 7, 2014, 12, 30, 15),
 (10, 7, 2014, 12, 30, 15),
 (10, 7, 2014, 12, 30, 15),
 (10, 7, 2014, 12, 30, 15)]

In [4]:
#2015, feb, 22

In [34]:
status3 = status2.filter(lambda x: x[2]==2015 and  x[3]==2 and x[4]>=22).map(lambda x: (x[0], x[1], x[5]))

status3.take(5)

[(10, 9, 23), (10, 9, 23), (10, 9, 23), (10, 8, 23), (10, 8, 23)]

In [35]:
stations.take(1)

['2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013']

In [37]:
stations2 = stations.map(lambda x: x.split(',')).filter(lambda x: x[5] == 'San Jose').map(lambda x: (int(x[0]), x[1]))
stations2.take(1)

[(2, 'San Jose Diridon Caltrain Station')]

In [38]:
status_kv = status3.keyBy(lambda x: x[0])
status_kv.take(1)

[(10, (10, 9, 23))]

In [39]:
stations_kv = stations2.keyBy(lambda x: x[0])
stations_kv.take(1)

[(2, (2, 'San Jose Diridon Caltrain Station'))]

In [40]:
joined = status_kv.join(stations_kv)

In [ ]:
# 2, (2, 7, 23) ('2', 'San Jose Diridon Caltrain Station')

In [41]:
joined.take(1)

[(2, ((2, 20, 0), (2, 'San Jose Diridon Caltrain Station')))]

In [42]:
cleaned = joined.map(lambda x: (x[0], x[1][0][1], x[1][0][2], x[1][1][1]))

In [43]:
cleaned.take(1)

[(2, 20, 0, 'San Jose Diridon Caltrain Station')]

In [ ]:
# [(station_id,bikes_available,hour,name),...]

In [59]:
avgbyhour = cleaned.keyBy(lambda x: (x[3], x[2])) \
    .mapValues(lambda x: (x[1], 1)) \

avgbyhour.take(1)

[(('San Jose Diridon Caltrain Station', 0), (20, 1))]

In [60]:
avgbyhour = cleaned.keyBy(lambda x: (x[3], x[2])) \
    .mapValues(lambda x: (x[1], 1)) \
    .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))

avgbyhour.take(1)

[(('San Jose Civic Center', 2), (3100, 420))]

In [61]:
avgbyhour = cleaned.keyBy(lambda x: (x[3], x[2])) \
    .mapValues(lambda x: (x[1], 1)) \
    .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
    .mapValues(lambda x: (x[0]/x[1]))

avgbyhour.take(1)

[(('San Jose Civic Center', 2), 7.380952380952381)]

In [62]:
3100/420

7.380952380952381

In [66]:
topavail = avgbyhour.keyBy(lambda x: x[1])
topavail.take(5)

[(7.380952380952381, (('San Jose Civic Center', 2), 7.380952380952381)),
 (8.30952380952381, (('Paseo de San Antonio', 13), 8.30952380952381)),
 (8.285714285714286, (('San Salvador at 1st', 10), 8.285714285714286)),
 (7.7023809523809526, (('Japantown', 4), 7.7023809523809526)),
 (3.7142857142857144,
  (('Santa Clara County Civic Center', 2), 3.7142857142857144))]

In [63]:
topavail = avgbyhour.keyBy(lambda x: x[1]) \
            .sortByKey(ascending=False) \
            .map(lambda x: (x[1][0], x[1][0][1], x[0]))

topavail.take(5)

[(('San Jose Diridon Caltrain Station', 17), 17, 16.590476190476192),
 (('San Jose Diridon Caltrain Station', 7), 7, 16.492857142857144),
 (('San Jose Diridon Caltrain Station', 6), 6, 16.34285714285714),
 (('San Jose Diridon Caltrain Station', 18), 18, 16.21904761904762),
 (('San Jose Diridon Caltrain Station', 19), 19, 15.64047619047619)]